# Import Library

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
# import matplotlib.pyplot as plt

from sklearn import cluster
from sklearn import mixture
from sklearn import preprocessing
from collections import Counter
# from mpl_toolkits.mplot3d import Axes3D

# Data Preprocess
### Reading data

In [5]:
def read_data(folder, step):
    file_path = "../data/50mph/" + str(folder) + "/" + str(folder) + str(step)

    data = pd.read_csv(file_path)
    
    return data
    

### Preprocess

In [86]:
def match_partnum(df1, df2):
    diff1 = list(set(df1["part_num"]) - set(df2["part_num"]))
    diff2 = list(set(df2["part_num"]) - set(df1["part_num"]))
    print(diff1, diff2)
    print(df1.shape, df2.shape)
    
    del_row_idx1 = []
    del_row_idx2 = []
    for i in range(len(diff1)):
        del_row_idx = int(np.where(df1["part_num"] == diff1[i])[0])
        df1 = df1.drop(del_row_idx, axis=0)
        
    for i in range(len(diff2)):
        del_row_idx2[i] = int(np.where(df2["part_num"] == diff2[i])[0])
        df2 = df2.drop(del_row_idx, axis=0)
        
    print(df1.shape, df2.shape)
    return df1, df2

# Clustering

In [85]:
folder_init, folder_final = "cb_d9out", "cb_d16out"
step_init, step_final = "_Step1.csv", "_Step303.csv"

init_state = read_data(folder_init, step_init)
final_state = read_data(folder_final, step_final)
print(init_state.shape)
print(final_state.shape)

init_state, final_state = match_partnum(init_state, final_state)

part_movement = []
part_movement = final_state.iloc[:,2:5] - init_state.iloc[:,2:5]
part_movement



(11428, 20)
(11426, 20)
[11360, 11284] []
(11428, 20) (11426, 20)
(11426, 20) (11426, 20)


,centroid_x,centroid_y,centroid_z
0,-7.880950e-04,2.445661e-04,2.610137e-05
1,-1.444983e-03,-1.513556e-04,2.489561e-04
2,-1.438779e-03,-5.801196e-04,1.194097e-04
3,2.480931e-05,-1.303955e-04,3.698886e-07
4,-2.670181e-03,-6.009136e-04,2.385055e-04
5,-1.575147e-03,-2.609706e-04,1.770458e-04
6,-2.986568e-03,-3.705369e-04,3.140084e-06
7,-5.103090e-03,1.887307e-04,-3.436783e-03
8,-4.546165e-03,-2.013134e-04,-5.399433e-04
9,-4.181552e-04,-6.790751e-04,-3.832574e-10
